# Подготовка среды

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# import shutil
# shutil.copytree("/content/drive/Shared drives/hse_BERT/hse_Af_Tr_BERT/embeddings/bert-base-multilingual-cased-polit3/bin", "/content/data/bin2")

In [3]:
import os
os.chdir('/content/drive/Shared drives/hse_BERT/hse_Af_Tr_BERT')

In [4]:
folder = "bert-base-multilingual-cased-polit3"

if not os.path.isdir(os.path.join("embeddings", folder)):
    os.mkdir(os.path.join("embeddings", folder))
if not os.path.isdir(os.path.join("embeddings", folder, "bin")):
    os.mkdir(os.path.join("embeddings", folder, "bin"))
if not os.path.isdir(os.path.join("embeddings", folder, "ready")):
    os.mkdir(os.path.join("embeddings", folder, "ready"))
if not os.path.isdir(os.path.join("embeddings", folder, "tests")):
    os.mkdir(os.path.join("embeddings", folder, "tests"))

In [5]:
import shutil
shutil.copytree("/content/drive/Shared drives/hse_BERT/hse_Af_Tr_BERT/embeddings/{}/bin".format(folder), "/content/data/bin2")

'/content/data/bin2'

In [6]:
!pip install transformers
!pip install razdel

     |████████████████████████████████| 2.1MB 8.3MB/s 
     |████████████████████████████████| 3.3MB 30.4MB/s 
     |████████████████████████████████| 901kB 40.1MB/s 


In [7]:
import torch
import nltk
import re
import json
import time
import numpy as np
import pandas as pd
import pickle as pc
from transformers import BertTokenizer, BertModel

import matplotlib.pyplot as plt
% matplotlib inline

from scipy.spatial.distance import cosine
from razdel import sentenize

In [8]:
def my_dump(file, name):
    with open("./" + name + ".pic", 'wb') as f:
        pc.dump(file, f)

def my_load(name):
    with open("./" + name + ".pic", 'rb') as f:
        return pc.load(f)

In [9]:
# Load pre-trained model tokenizer (vocabulary)
# 'DeepPavlov/rubert-base-cased-sentence'
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [10]:
model = BertModel.from_pretrained('bert-base-multilingual-cased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
         

#text processing

## generate (split) sentences

In [ ]:
%%time
# name = "all_cyberleninka_polit2"
name = "all_cyberleninka_liter2"
with open("./texts/{}/{}.txt".format(name, name), 'r') as f:
    data = f.read()
sent2 = list(sentenize(data))
my_dump(sent2, name)

CPU times: user 6min 1s, sys: 12.6 s, total: 6min 14s
Wall time: 6min 48s


#getting embeddings

## read sentences

In [10]:
%%time
name = "bin/all_cyberleninka_polit2"
# name = "bin/all_cyberleninka_liter2"
sent1 = my_load(name)

CPU times: user 9.61 s, sys: 4.44 s, total: 14 s
Wall time: 18 s


In [11]:
def get_text_from_ST(ss):
    return ss.text
sent = list(map(get_text_from_ST, sent1))
len(sent)

2348859

In [12]:
del sent1

##process

In [13]:
def sum_tensors(token):
    return torch.sum(token[-4:], dim=0)

def get_embeddings(text):
    tokenized_text = tokenizer.tokenize("[CLS] " + text.lower() + " [SEP]")
    if len(tokenized_text) > 80:
        return [],[]
    marked_text = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(tokenized_text)

    tokens_tensor = torch.tensor([marked_text])
    segments_tensors = torch.tensor([segments_ids])

    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        hidden_states = outputs[2]

    token_embeddings = torch.squeeze(torch.stack(hidden_states, dim=0),
                                    dim=1).permute(1,0,2)
    
    token_vecs_sum = list(map(sum_tensors, token_embeddings))
    return tokenized_text, token_vecs_sum

# token_vecs = hidden_states[-2][0]
# sentence_embedding = torch.mean(token_vecs, dim=0)

In [14]:
# text = "Тюмень стала самым «счастливым» городом России по уровню довольных своим городом людей."

def sum_tensors(token):
    return torch.sum(token[-4:], dim=0)

def get_embeddings2(text):
    tokenized_text = tokenizer.tokenize("[CLS] " + text.lower() + " [SEP]")
    if len(tokenized_text) > 80:
        return [],[]
    marked_text = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(tokenized_text)

    tokens_tensor = torch.tensor([marked_text])
    segments_tensors = torch.tensor([segments_ids])

    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        hidden_states = outputs[2]

    token_embeddings = torch.squeeze(torch.stack(hidden_states, dim=0),
                                    dim=1).permute(1,0,2)

    token_vecs_sum = list(map(sum_tensors, token_embeddings))

    new_tokens = []
    new_token_vecs = []
    j = -1
    for i in range(len(tokenized_text)):
        if len(tokenized_text[i]) > 2 and tokenized_text[i][0] == "#" and tokenized_text[i][1] == "#":
            new_tokens[j] = new_tokens[j] + tokenized_text[i][2:]
            new_token_vecs[j].append(token_vecs_sum[i])
        else:
            new_tokens.append(tokenized_text[i])
            new_token_vecs.append([token_vecs_sum[i]])
            j+=1
    new_token_vecs2 = []
    for i in new_token_vecs:
        new_token_vecs2.append(sum(i))
    return new_tokens, new_token_vecs2

In [15]:
def get_batch_number(from_i = 0, to_i = 1000):
    
    files_r = os.listdir("./embeddings/{}/bin".format(folder))
    cut_files_r = []
    for i in files_r:
        if i.split('.')[-1] == 'pic':
            cut_files_r.append(int(i.split('.')[0].split('_')[1].split(" ")[0]))
    diff = set(range(from_i, to_i)) - set(cut_files_r)
    return sorted(list(diff))

def reserve_batch(number):
    print("reserve {}".format(number))
    my_dump("file", "./embeddings/{}/bin/{}_{:05d}_empty".format(folder,batch, number))

batch = 1000
from_i = 0
to_i = 1000
start_time = time.time()

l = get_batch_number( from_i, to_i)
while len(l) > 0:
    j = l[-4]
    reserve_batch(j)
    pr_tr = {}
    temp_time = time.time()
    for i in range(j * batch, (j + 1) * batch):
        tt, tvs = get_embeddings2(sent[i])
        for i in range(len(tt)):
            if tt[i].isalpha():# and len(tt[i]) > 2 :
                if tt[i] in pr_tr:
                    pr_tr[tt[i]].append(tvs[i])
                else:
                    pr_tr[tt[i]] = [tvs[i]]

    with open("./embeddings/{}/bin/{}_{:05d}.pic".format(folder, batch, j), 'wb') as f:
        pc.dump(pr_tr, f)
    # res.update(pr_tr)

    t = time.time()
    print("done {:05d} --- time: {}\t--- total: {}".format(j, t - temp_time, t - start_time))

    l = get_batch_number( from_i, to_i)

reserve 994
done 00994 --- time: 143.65636205673218	--- total: 143.68875002861023
reserve 991
done 00991 --- time: 147.92172813415527	--- total: 291.649879693985
reserve 988
done 00988 --- time: 154.40953922271729	--- total: 446.1133484840393
reserve 984
done 00984 --- time: 141.88636088371277	--- total: 588.0364015102386
reserve 981
done 00981 --- time: 143.60099148750305	--- total: 731.6678831577301
reserve 978
done 00978 --- time: 148.19910168647766	--- total: 879.8986964225769
reserve 975
done 00975 --- time: 141.57722735404968	--- total: 1021.504998922348
reserve 972
done 00972 --- time: 144.83072328567505	--- total: 1166.3604872226715
reserve 970
done 00970 --- time: 146.43743920326233	--- total: 1312.8312292099
reserve 967
done 00967 --- time: 146.41510033607483	--- total: 1459.2718436717987
reserve 964
done 00964 --- time: 158.9853105545044	--- total: 1618.285499572754
reserve 961
done 00961 --- time: 141.54458498954773	--- total: 1759.8650116920471
reserve 958
done 00958 --- t

IndexError: ignored

In [ ]:
# %%time
# batch = 1000
# start_time = time.time()

# ftom_b = 224
# to_b = 500

# print("senteces from {} to {}".format(ftom_b * batch, to_b * batch))

# for j in range(ftom_b, to_b):
#     pr_tr = {}
#     temp_time = time.time()
#     for i in range(j * batch, (j + 1) * batch):
#         tt, tvs = get_embeddings2(sent[i])
#         for i in range(len(tt)):
#             if tt[i].isalpha() and len(tt[i]) > 2 :
#                 if tt[i] in pr_tr:
#                     pr_tr[tt[i]].append(tvs[i])
#                 else:
#                     pr_tr[tt[i]] = [tvs[i]]

#     with open("./embeddings/{}/bin/{}_{:05d}.pic".format(folder, batch, j), 'wb') as f:
#         pc.dump(pr_tr, f)

#     t = time.time()
#     print("done {:05d} --- time: {}\t--- total: {}".format(j, t - temp_time, t - start_time))

# # for i in pr_tr:
# #     print(i, len(pr_tr[i]))

#aggregation of embeddings M



In [ ]:
%%time
count_t = {}
sem_emb = {}

for name in cut_files:
    with open("./embeddings/{}/bin/{}".format(folder, name), 'rb') as f:
        data = pc.load(f)
    for i in data.keys():
        if i not in count_t:
            count_t[i] = 0
            sem_emb[i] = data[i]
        else:
            count_t[i] += 1
            sem_emb[i] = sem_emb[i] + data[i]

res = {}
for key in count_t:
    res[key] = sem_emb[key] / count_t[key]
    # print(name)

In [ ]:
with open("./embeddings/{}/dict_{}.pic".format(folder, folder), "bw") as f:
    pc.dump(res, f)

#aggregation of embeddings



In [ ]:
# folder = "bert-base-multilingual-cased-liter"

##list of files

In [ ]:
files = os.listdir("./embeddings/{}/bin".format(folder))
cut_files = []
for i in files:
    if i.split('.')[-1] == 'pic':
        cut_files.append(i)

##get count

In [ ]:
%%time
from_files = {}
all_count = {}

for name in cut_files:
    with open("./embeddings/{}/bin/{}".format(folder, name), 'rb') as f:
        data = pc.load(f)
    for i in data.keys():
        if i not in all_count:
            all_count[i] = len(data[i])
            from_files[i] = [name]
        else:
            all_count[i] += len(data[i])
            from_files[i].append(name)
    # print(name)


CPU times: user 5min 36s, sys: 12.6 s, total: 5min 49s
Wall time: 9min 39s


In [ ]:
my_dump(from_files, "embeddings/{}/from_files".format(folder))
my_dump(all_count, "embeddings/{}/all_count".format(folder))

## aggregate

In [11]:
from_files = my_load("embeddings/{}/from_files".format(folder))
all_count = my_load("embeddings/{}/all_count".format(folder))

In [12]:
out = list(zip(all_count.keys(), list(all_count.values())))
out.sort(key = lambda i: i[1], reverse=True)
# out[:20]

In [13]:
keys = list(all_count.keys())
k1 = []
k2 = []
k3 = []
kb = []
l = 0
for i in keys:
    l = len(i)
    if l == 1:
        k1.append(i)
    elif l == 2:
        k2.append(i)
    elif l == 3:
        k3.append(i)
    else:
        kb.append(i)
len(k1),len(k2),len(k3),len(kb)

(82, 1455, 7702, 336472)

In [14]:
all_words = []
for i in kb:
    if all_count[i] > 10:
        all_words.append(i)
len(all_words)

54176

In [15]:
def check_existing_emb(words):
    d = dict(out)
    fl = True
    for i in words:
        if i not in d:
            print("{} not in dictionary".format(i))
            fl = False
    return fl

def get_all_embeddings(words):
    all_files = []
    embeddings = {}
    for w in words:
        embeddings[w] = []

    for i in words:
        all_files += from_files[i]

    for name in set(all_files):
        with open("/content/data/bin2/{}".format( name), 'rb') as f:
            data = pc.load(f)
        for w in words:
            if w in data:
                embeddings[w] += data[w]
    return embeddings

def sum_embeddings(embeddings):
    res = {}
    res_emb = {}
    for i in embeddings:
        res[i] = torch.empty(len(embeddings[i]), embeddings[i][0].size(0))
        for j in range(len(embeddings[i])):
            res[i][j] = embeddings[i][j]
        res_emb[i] = res[i].mean(dim = 0)
    return res_emb

In [16]:
# batch = 1000
# st_time = time.time()

# ftom_b = 47
# to_b = 56

# print("words from {} to {}, max batchs = {}".format(ftom_b * batch, to_b * batch, len(all_words)//batch))

# all_emb = {}
# for b in range(ftom_b, to_b):
#     t_time = time.time()
#     words = all_words[b*batch:(b+1)*batch]
#     emb = get_all_embeddings(words)
#     res_emb = sum_embeddings(emb)
    
#     with open("./embeddings/{}/ready/{}_{:05d}.pic".format(folder,batch, b), 'wb') as f:
#         pc.dump(res_emb, f)

#     print("done: {} --- time: {}\t--- total: {}".format(b,time.time() - t_time, time.time() - st_time))

In [17]:
def get_batch_number(max_i, from_i = 0):
    
    files_r = os.listdir("./embeddings/{}/ready".format(folder))
    cut_files_r = []
    for i in files_r:
        if i.split('.')[-1] == 'pic':
            cut_files_r.append(int(i.split('.')[0].split('_')[1]))
    diff = set(range(from_i, max_i)) - set(cut_files_r)
    return sorted(list(diff))

def reserve_batch(number):
    print("reserve {}".format(b))
    my_dump("file", "./embeddings/{}/ready/{}_{:05d}_empty".format(folder,batch, b))

batch = 1000
from_i = 0

st_time = time.time()
print(" max batchs = {}".format( len(all_words)//batch))

all_emb = {}
l = get_batch_number(len(all_words)//batch + 1, from_i)
while len(l) > 0:
    b = l[-1]
    reserve_batch(b)
    t_time = time.time()
    words = all_words[b*batch:(b+1)*batch]
    emb = get_all_embeddings(words)
    res_emb = sum_embeddings(emb)
    
    with open("./embeddings/{}/ready/{}_{:05d}.pic".format(folder,batch, b), 'wb') as f:
        pc.dump(res_emb, f)

    print("done: {} --- time: {}\t--- total: {}".format(b,time.time() - t_time, time.time() - st_time))
    l = get_batch_number(len(all_words)//batch + 1, from_i)

 max batchs = 54
reserve 54
done: 54 --- time: 160.6246566772461	--- total: 160.6375663280487
reserve 52
done: 52 --- time: 670.3634383678436	--- total: 831.0146355628967
reserve 51
done: 51 --- time: 712.3574945926666	--- total: 1543.3936591148376
reserve 49
done: 49 --- time: 751.918210029602	--- total: 2295.3341369628906
reserve 47
done: 47 --- time: 781.9779424667358	--- total: 3077.326022386551
reserve 46
done: 46 --- time: 787.021909236908	--- total: 3864.368085384369
reserve 44
done: 44 --- time: 800.557874917984	--- total: 4664.943334579468
reserve 42
done: 42 --- time: 821.7799801826477	--- total: 5486.739743709564
reserve 40
done: 40 --- time: 819.4751694202423	--- total: 6306.236320972443
reserve 38
done: 38 --- time: 829.1979556083679	--- total: 7135.449824810028
reserve 36
done: 36 --- time: 845.8310024738312	--- total: 7981.301098823547
reserve 34
done: 34 --- time: 844.6680612564087	--- total: 8825.98593711853
reserve 32
done: 32 --- time: 850.0107591152191	--- total: 96

#building dictionary


In [ ]:
folder = "bert-base-multilingual-cased-liter"

In [ ]:
files = os.listdir("./embeddings/{}/ready".format(folder))
res = dict()
for i in files:
    with open("./embeddings/{}/ready/{}".format(folder,i), "rb") as f:
        temp = pc.load(f)
        # print(type(temp))
        res.update(temp)

In [ ]:
with open("./embeddings/{}/dict_{}.pic".format(folder, folder), "bw") as f:
    pc.dump(res, f)

In [ ]:
with open("./embeddings/{}/dict_{}.json".format(folder, folder), 'w', encoding="utf-8") as f:
    json.dump(lista_items, f, indent=4)

#Tests

##preparation

In [ ]:
# folder = "bert-base-multilingual-cased-liter"

In [ ]:
with open("./embeddings/{}/dict_{}.pic".format(folder, folder), "br") as f:
    dct1  = pc.load(f)

# with open("/content/drive/Shareddrives/hse_BERT/hse_Af_Tr_BERT/embeddings/sent1/all_emc.pic", "br") as f:
#     dct1  = pc.load(f)

In [ ]:
def get_word(emb, dct, nin = []):
    word = "none"
    cs = -100 
    for key in dct:
        temp_cs = 1 - cosine(emb, dct[key])
        if temp_cs > cs and key not in nin:
            cs = temp_cs
            word = key
    return word, cs

def get_word_m(emb, dct, count = 3):
    res = {}
    for key in dct:
        res[key] = 1 - cosine(emb, dct[key])
    l = list(res.items()) 
    l.sort(key= lambda x:x[1], reverse = True)
    return dict(l[:count])
def add_names(df):
    names = {': capital-common-countries' : "Capitals-common-countries",
 ': capital-world': "Capital-countries",
 ': currency': "Currency",
#  ': city-in-state',
 ': family': "Family",
 ': gram1-adjective-to-adverb': "Adjective adverb",
 ': gram2-opposite': "Opposite",
 ': gram3-comparative': "Comparative",
#  ': gram4-superlative',
#  ': gram5-present-participle',
 ': gram6-nationality-adjective': "Nationality adjective",
#  ': gram7-past-tense',
#  ': gram8-plural',
#  ': gram9-plural-verbs'
    }
    df["type"] = "none"
    for i in names:
        if i in df.index:
            df.loc[i, "type"] = names[i]

    return df

def add_names_rus(df):
    names = {': capital-common-countries' : "Известные столицы - страны",
 ': capital-world': "Столицы - страны",
 ': currency': "Валюта",
#  ': city-in-state',
 ': family': "Семья",
 ': gram1-adjective-to-adverb': "Прилагательное - наречие",
 ': gram2-opposite': "Антонимы",
 ': gram3-comparative': "Сравнение",
#  ': gram4-superlative',
#  ': gram5-present-participle',
 ': gram6-nationality-adjective': "Национальность - прилагательное",
#  ': gram7-past-tense',
#  ': gram8-plural',
#  ': gram9-plural-verbs'
    }
    df["type"] = "none"
    for i in names:
        if i in df.index:
            df.loc[i, "type"] = names[i]

    return df

In [ ]:
# get_word_m(dct1["машина"], dct1, 10)

##GATS

###generate test

In [ ]:
with open("/content/drive/Shareddrives/hse_BERT/hse_Af_Tr_BERT/tests/questions-words.txt", "r") as f:
    lines = f.read().splitlines()

words = {}
words2 = {}
all_w = []
key = "no_key"
for i in lines:
    m = i.split(" ")
    if i[0] == ":":
        key = i
    else:
        if key not in words:
            words[key] = []
            words2[key] = []
        words[key].append(m)
        all_w = all_w + m
        words2[key].append((m[0], m[1]))
        words2[key].append((m[2], m[3]))
all_w = set(all_w)
for key in words2:
    words2[key] = set(words2[key])

In [ ]:
names = [
        #  ': capital-common-countries',
 ': capital-world',
#  ': currency',
#  ': city-in-state',
 ': family',
 ': gram1-adjective-to-adverb',
 ': gram2-opposite',
 ': gram3-comparative',
#  ': gram4-superlative',
#  ': gram5-present-participle',
 ': gram6-nationality-adjective',
#  ': gram7-past-tense',
#  ': gram8-plural',
#  ': gram9-plural-verbs'
 ]

In [ ]:
words3 = {}
find_words = []
for key in names:
    df = pd.read_csv("tests/gats/{}.csv".format(key[2:]),sep = ";", encoding = "cp1251")
    d1 = list(zip(df["eng_w1"], df["rus_w1"]))
    d2 = list(zip(df["eng_w2"], df["rus_w2"]))
    words3[key] = dict(d1)
    words3[key].update(dict(d2))
    all_rus = [*df["rus_w1"], *df["rus_w2"]]

In [ ]:
def translate_sl(m ,sl):
    m2 = []
    for i in m:
        m2.append(sl[i])
    return m2

In [ ]:
words_rus = {}
for key in names:
    words_rus[key] = []
    for i in words[key]:
        words_rus[key].append(translate_sl(i ,words3[key]))

In [ ]:
with open(os.path.join("embeddings", folder, "tests", "ready.pic"), "bw") as f:
    pc.dump(words_rus, f)

##get quality


In [ ]:
with open("tests/gats/ready.pic", "br") as f:
    words_rus = pc.load( f)

In [ ]:
def cut_b(text):
    return tokenizer.tokenize("[CLS] " + text.lower() + " [SEP]")[1]

###3cosadd

In [ ]:
def cos_add(dct):
    res_df = pd.DataFrame(columns=["w1","w2","w3", "expected", "prdicted", "cosine", "group"])
    for key in words_rus:
        for m_l in words_rus[key]:
            m = []
            for i in m_l:
                m.append(cut_b(i))
        for m in words_rus[key]:
            if m[0] in dct and m[1] in dct and m[2] in dct and m[3] in dct:
                pred, prob = get_word(dct[m[0]] - dct[m[1]] + dct[m[3]], dct, nin = [m[0], m[1], m[3]])
                res_df.loc[res_df.shape[0]] = [m[0], m[1], m[3], m[2], pred, prob, key]
                # print(m[2], pred)
    return res_df

In [ ]:
res_df1 = cos_add(dct1)

In [ ]:
# res_df1[res_df1["group"] == ": gram2-opposite"]

In [ ]:
res_df1["group"].unique()

array([': family', ': gram3-comparative'], dtype=object)

In [ ]:
res_df1["result"] = res_df1["expected"]  == res_df1["prdicted"] 
count_1 = res_df1.groupby("group").count()
res_sort_1 = res_df1.groupby("group").mean().sort_values(by = "result")
res_sort_1["count"] = count_1["w1"]
res_sort_1 = add_names(res_sort_1)
res_sort_1.index = res_sort_1["type"]
res_sort_1[["cosine", "result", "count"]]

,cosine,result,count
type,,,
Comparative,0.895160,False,6
Family,0.900485,True,2


In [ ]:
# dct1[]

In [ ]:
# cut_b("камбоджа")

'кам'

###3cosavg

In [ ]:
def cos_avg(dct):
    res = pd.DataFrame(columns=["w1","p1","w2","p2","prob", "key","count"])
    for key in words_rus:
        res_df = pd.DataFrame(columns=["w1","w2","w3", "expected", "group"])
        for m_l in words_rus[key]:
            m = []
            for i in m_l:
                
                    m.append(cut_b(i))
                # else:
                #     m.append("none")
        for m in words_rus[key]:
            if m[0] in dct and m[1] in dct and m[2] in dct and m[3] in dct:
                res_df.loc[res_df.shape[0]] = [dct[m[0]].numpy(), 
                                               dct[m[1]].numpy(), 
                                               dct[m[3]].numpy(), 
                                               dct[m[2]].numpy(),
                                               key]
                # print(m[2], pred)
            
            
        w1,p1 = get_word(res_df["w1"].mean() - res_df["w2"].mean() + res_df["w3"].mean(), dct)
        w2,p2 = get_word(res_df["expected"].mean(), dct)
        prob = 1 - cosine(res_df["w1"].mean() - res_df["w2"].mean() + res_df["w3"].mean(), res_df["expected"].mean())
        res.loc[res.shape[0]] = [w1,p1,w2,p2, prob, key, res_df.shape[0]]
    return res

In [ ]:
res_df1_avg = cos_avg(dct1)
res_df1_avg.index = res_df1_avg["key"]
add_names(res_df1_avg)
res_df1_avg.index = res_df1_avg["type"]
res_df1_avg[["w1", "p1", "w2", "p2", "prob", "count"]]

,w1,p1,w2,p2,prob,count
type,,,,,,
Capital-countries,none,-100,none,-100,NaN,0
Family,отец,0.969902,отец,0.969902,1,2
Adjective adverb,none,-100,none,-100,NaN,0
Opposite,none,-100,none,-100,NaN,0
Comparative,большой,0.982492,большой,0.982492,1,6
Nationality adjective,none,-100,none,-100,NaN,0


###3cosmul


In [ ]:
def get_word_mul(w1,w2,w3, dct):
    word = "none"
    cs = -100 
    for key in dct:
        e1 = 1 - cosine(dct[w1], dct[key])
        e2 = 1 - cosine(dct[w2], dct[key])
        e3 = 1 - cosine(dct[w3], dct[key])
        if e1 - e2 + e3 > cs and key != w1 and key != w2 and key != w3:
            cs = e1 - e2 + e3
            word = key
    return word, cs

In [ ]:
def cos_mul(dct):
    res_df = pd.DataFrame(columns=["w1","w2","w3", "expected", "prdicted", "cosine", "group"])
    for key in words_rus:
        for m_l in words_rus[key]:
            m = []
            for i in m_l:
                
                m.append(cut_b(i))
                # else:
                #     m.append("none")
        for m in words_rus[key]:
            if m[0] in dct and m[1] in dct and m[2] in dct and m[3] in dct:
                pred, prob = get_word_mul(m[0], m[1], m[3], dct)
                res_df.loc[res_df.shape[0]] = [m[0], m[1], m[3], m[2], pred, prob, key]
                # print(m[2], pred)
    return res_df

In [ ]:
res_df1_mul = cos_mul(dct1)
res_df1_mul["result"] = res_df1_mul["expected"]  == res_df1_mul["prdicted"] 
count_1 = res_df1_mul.groupby("group").count()
res_df_1_mul = res_df1_mul.groupby("group").mean().sort_values(by = "result")
res_df_1_mul["count"] = count_1["w1"]
res_df_1_mul = add_names(res_df_1_mul)
res_df_1_mul.index = res_df_1_mul["type"]
res_df_1_mul[["cosine", "result", "count"]]

,cosine,result,count
type,,,
Comparative,0.930116,False,6
Family,0.942573,True,2


In [ ]:
# res_df1_mul

###mix table

In [ ]:
ddd1 = pd.DataFrame()
def get_r1(r1, metric):
    r1["metric"] = metric
    return r1.pivot(columns = ["metric"])

In [ ]:
ddd1 = get_r1(res_sort_1[["cosine", "result", "count"]], "3CosAdd")
ddd1 = ddd1.merge(get_r1(res_df_1_mul[["cosine", "result", "count"]], "3CosMul"), on = "type")
r1 = res_df1_avg[["prob", "count"]].copy()
r1["cosine"] = r1["prob"]
ddd1 = ddd1.merge(get_r1(r1[["cosine", "count"]], "3CosAvg"), on = "type")
ddd1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,cosine,result,count,cosine,result,count,cosine,count
metric,3CosAdd,3CosAdd,3CosAdd,3CosMul,3CosMul,3CosMul,3CosAvg,3CosAvg
type,,,,,,,,
Comparative,0.895160,False,6,0.930116,False,6,1,6
Family,0.900485,True,2,0.942573,True,2,1,2


In [ ]:
ddd1[["result", "cosine", "count"]]

result            cosine                     count                
metric      3CosAdd 3CosMul   3CosAdd   3CosMul 3CosAvg 3CosAdd 3CosMul 3CosAvg
type                                                                           
Comparative   False   False  0.895160  0.930116       1       6       6       6
Family         True    True  0.900485  0.942573       1       2       2       2